# Problem 1

Generate graph $H^n_{m,a}$ in the Buckley-Ostgus model and analyse the dependence of the loops count on parameter $a$ value

In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def generateSimpleBuckleyOsthusGraph(a,n):
    G = nx.MultiDiGraph()
    G.add_node(0)
    G.add_edge(0,0)
    p = [a+1]
    while G.number_of_nodes() < n:
        new_node = G.number_of_nodes()
        G.add_node(new_node)
        p.append(a)
        probabilities = np.array(p)/((a+1.0)*new_node+a)
        destination = np.random.choice(G.nodes(),1,p=probabilities)[0]
        G.add_edge(new_node,destination)
        p[destination] += 1
    return G

In [ ]:
def generateBuckleyOsthusGraph(a, n, m):
    G1mn = generateSimpleBuckleyOsthusGraph(a, m*n)
    G = nx.MultiDiGraph()
    for u, v in G1mn.edges():
        u_new, v_new = u // m, v // m
        G.add_edge(u_new, v_new)
    return G

In [ ]:
H = generateBuckleyOsthusGraph(1000,1000,1)

In [ ]:
H.number_of_nodes()

In [ ]:
H.number_of_edges()

In [ ]:
edges = H.edges()
loops = [out_vert for (out_vert,in_vert) in list(edges) if out_vert == in_vert]
len(loops)

In [ ]:
loops_counts = []
a_list = [0.01,0.1,0.2,0.5,1,10]
Niter = 20
for a in a_list:
    loops_tmp = 0
    for iter in range(Niter):
        H = generateBuckleyOsthusGraph(a,1000,1)
        edges = H.edges()
        loops = [out_vert for (out_vert,in_vert) in list(edges) if out_vert == in_vert]
        loops_tmp += len(loops)
    loops_counts.append(loops_tmp / Niter)

In [ ]:
plt.plot(a_list,loops_counts)

# Problem 2

Generate Backley-Ostgus graph $H_{a,m}^n$ with $n=5000$ nodes and $m=2$, $a=0.27$. In this graph calculate assortativity and plot $d_{nn}$.

Approximate the result $d_{nn} \sim d^\delta$

In [ ]:
H = generateBuckleyOsthusGraph(0.27, 5000, 2)

Value $X(d_1,d_2)$ is the number of edges between the nodes with degrees $d_1$ and $d_2$ in $G$.
Assortativity
    $$
    d_{nn}(d)=\frac{\sum\limits_{d_1}d_1X(d_1,d)}{\sum\limits_{d_1}X(d_1,d)}.
    $$

In [ ]:
from collections import defaultdict

def get_X(G):
    result = defaultdict(int)
    for u, v in G.edges():
        result[(G.degree(u), G.degree(v))] += 1
        result[(G.degree(v), G.degree(u))] += 1
    return result

In [ ]:
degrees = set(dict(H.degree()).values())
X = get_X(H)
degs = []
assorts = []

for d in sorted(degrees):
    dnn = sum([d1 * X[(d1, d)] for d1 in degrees]) / sum([X[(d1, d)] for d1 in degrees])
    degs.append(d)
    assorts.append(dnn)

plt.loglog(degs, assorts, ls='None', marker='o')
plt.show()

In [ ]:
def approx_assort(d, delta, c):
    return c / d**delta

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(approx_assort, degs, assorts)

In [ ]:
delta, c = popt

In [ ]:
plt.loglog(degs, assorts, ls='None', marker='o')
plt.loglog(degs,[approx_assort(d, delta, c) for d in degs], ls='--',color="red")
plt.xlabel("Degree, $d$")
plt.ylabel("Mean neighbors' degree, $d_{nn}$")
plt.show()

# Problem 3

Analyze assortativity of the Erdos-Renyi graph 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.erdos_renyi_graph(1000,0.2)

For undirected graph without loops and multiple edges we can use the method from networkx library

In [ ]:
d_nn = nx.average_degree_connectivity(G)
d_nn

In [ ]:
plt.scatter(d_nn.keys(),d_nn.values())
plt.ylim(top=max(d_nn.values())*1.01, bottom = min(d_nn.values())*0.99)
plt.xlabel('Degree')
plt.ylabel("Average nearest neighbours' degree")

# Problem 4 

Analyze the dependence between assortativity and parameter $a$ value in the Backley-Ostgus model with $n = 2000, m = 2$

In [ ]:
def calc_assorts(G):
    degrees = set([value for (key,value) in G.degree()])
    X = get_X(G)
    assort_dict = {}
    for d in sorted(degrees):
        dnn = float(sum(d1 * X[(d1,d)] for d1 in degrees))/sum(X[(d1,d)] for d1 in degrees)
        assort_dict[d] = dnn    
    return assort_dict

In [ ]:
n = 2000
m = 2

In [ ]:
for i in range(1,40,5):
    a = i/10
    print(f'a = {a}')
    G_tmp = generateBuckleyOsthusGraph(a,n,m)
    assort_dict = calc_assorts(G_tmp)
    
    plt.loglog(assort_dict.keys(),assort_dict.values(),ls='None',marker='o',label=a)
plt.legend(title = "Values of a")
plt.show()

In [ ]:
a = 6

G_tmp = generateBuckleyOsthusGraph(a,n,m)
assort_dict = calc_assorts(G_tmp)
    
plt.loglog(assort_dict.keys(),assort_dict.values(),ls='None',marker='o',label=a)
plt.legend(title = "Values of a")
plt.show()